### First level analysis codes for BIDS, modified from Russ Poldrack

In [7]:
# Set up
task_name = 'ant2'
task_cond = ['c1-alert','c2-orient','c3-executive']
sess_name = 's02'

fig_plv = 0.05
fig_cor = 'fpr'
fig_in_nb = 'no'

data_dir = '/home/***/Projects/data_CBD/2018_***_DevBrainFuncAtlas'
sub_file = '/home/***/Projects/data_CBD/2018_***_DevBrainFuncAtlas/codes/UnivarActi/age_adult_t1_ant_00.csv'

#### The following sections do not need to be modified

In [8]:
# Import the required library
%matplotlib inline
%config InlineBackend.figure_format = "retina"

import numpy as np
import pandas as pd
import os, json, glob
import nilearn.plotting
import matplotlib.pyplot as mplt
import nipype.algorithms.modelgen as model

from bids import BIDSLayout
from nipype.caching import Memory
from nipype.interfaces import fsl
from nipype.interfaces.base import Bunch
from nistats.thresholding import map_threshold

In [9]:
# Check whether a temporary folder exists
temp_dir = os.path.join(data_dir, 'derivatives', 'temp')
if not os.path.exists(temp_dir): os.makedirs(temp_dir)
mem = Memory(base_dir = temp_dir)

# Check whether results folder exists
lv1res_dir = os.path.join(data_dir, 'derivatives', 'modeling')
if not os.path.exists(lv1res_dir): os.makedirs(lv1res_dir)
    
# Check whether figures folder exists
figure_dir = os.path.join(data_dir, 'derivatives', 'figures')
if not os.path.exists(figure_dir): os.makedirs(figure_dir)

In [10]:
# Acquire image data from BIDS
# layout = BIDSLayout(os.path.join(data_dir, 'datasets'))
# imgfiles = layout.get(suffix = "bold", task = 'em', session = 's02', extension = "nii.gz")

In [11]:
# Repeat for all subjects
sublist = pd.read_csv(sub_file, names = ['id'])

# copes = {}
zmaps = {}
for isub in sublist['id']:
    
    events = pd.read_csv(os.path.join(
        data_dir, "datasets", "sub-%s"%isub, "ses-%s"%sess_name, "func", 
        "sub-%s_ses-%s_task-%s_events.tsv"%(isub, sess_name, task_name)), sep = "\t")
    
    confounds = pd.read_csv(os.path.join(
        data_dir, "derivatives", "fmriprep", "sub-%s"%isub, "ses-%s"%sess_name, "func", 
        "sub-%s_ses-%s_task-%s_desc-confounds_regressors.tsv"%(isub, sess_name, task_name)), 
                            sep = "\t", na_values = "n/a")
    
    tskdsg = [Bunch(
        conditions = ['no', 
                      'double', 
                      'center', 
                      'spatial', 
                      'congruent', 
                      'incongruent'], 
        onsets = [list(events[events.trial_type == 'no'].onset), 
                  list(events[events.trial_type == 'double'].onset), 
                  list(events[events.trial_type == 'center'].onset), 
                  list(events[events.trial_type == 'spatial'].onset), 
                  list(events[events.trial_type == 'congruent'].onset), 
                  list(events[events.trial_type == 'incongruent'].onset)], 
        durations = [list(events[events.trial_type == 'no'].duration), 
                     list(events[events.trial_type == 'double'].duration), 
                     list(events[events.trial_type == 'center'].duration), 
                     list(events[events.trial_type == 'spatial'].duration), 
                     list(events[events.trial_type == 'congruent'].duration), 
                     list(events[events.trial_type == 'incongruent'].duration)], 
        regressors = [list(confounds.csf.fillna(0)), 
                      list(confounds.white_matter.fillna(0))], 
        regressor_names = ['csf', 'white_matter'])]
    
    skip = mem.cache(fsl.ExtractROI)
    skip_res = skip(in_file = os.path.join(
        data_dir, "derivatives", "fmriprep", "sub-%s"%isub, "ses-%s"%sess_name, "func", 
        "sub-%s_ses-%s_task-%s_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz"%(isub, sess_name, task_name)), 
                    t_min = 0, t_size = -1)
    
    json_file = os.path.join(
        data_dir, "datasets", "sub-%s"%isub, "ses-%s"%sess_name, "func", 
        "sub-%s_ses-%s_task-%s_bold.json"%(isub, sess_name, task_name))
    with open(json_file, 'r') as load_file: file_dic = json.load(load_file)
    
    specify = model.SpecifyModel()
    specify.inputs.input_units = 'secs'
    specify.inputs.functional_runs = skip_res.outputs.roi_file
    specify.inputs.time_repetition = file_dic['RepetitionTime']
    specify.inputs.high_pass_filter_cutoff = 128.
    specify.inputs.subject_info = tskdsg
    specify_model_results = specify.run()
    
    cond_alert = ['alert', 'T', ['no', 'double'], [-1, 1]]
    cond_orient = ['orient', 'T', ['center', 'spatial'], [-1, 1]]
    cond_executive = ["executive", 'T', ['congruent', 'incongruent'], [-1, 1]]
    contrasts = [cond_alert, cond_orient, cond_executive]
    
    level1_design = mem.cache(fsl.model.Level1Design)
    level1_design_res = level1_design(
        interscan_interval = file_dic['RepetitionTime'], 
        bases = {'dgamma':{'derivs': True}}, 
        session_info = specify_model_results.outputs.session_info, 
        model_serial_correlations = True, 
        contrasts = contrasts)
    
    modelgen = mem.cache(fsl.model.FEATModel)
    modelgen_res = modelgen(
        fsf_file = level1_design_res.outputs.fsf_files, 
        ev_files = level1_design_res.outputs.ev_files)
    
    mask = mem.cache(fsl.maths.ApplyMask)
    mask_res = mask(in_file = skip_res.outputs.roi_file, 
                    mask_file = os.path.join(
                        data_dir, "derivatives", "fmriprep", "sub-%s"%isub, "ses-%s"%sess_name, "func", 
                        "sub-%s_ses-%s_task-%s_space-MNI152NLin6Asym_desc-brain_mask.nii.gz"%(isub, sess_name, task_name)))
    
    filmgls = mem.cache(fsl.FILMGLS)
    filmgls_res = filmgls(in_file = mask_res.outputs.out_file, 
                          design_file = modelgen_res.outputs.design_file, 
                          tcon_file = modelgen_res.outputs.con_file, 
                          fcon_file = modelgen_res.outputs.fcon_file, 
                          autocorr_noestimate = True)
    
    # copes[isub] = list(filmgls_res.outputs.copes)
    zmaps[isub] = list(filmgls_res.outputs.zstats)

200619-21:59:37,351 nipype.workflow INFO:
	 [Node] Setting-up "7157e2e3c16071cf6fe61ec267976d25" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-utils-ExtractROI/7157e2e3c16071cf6fe61ec267976d25".
200619-21:59:37,353 nipype.workflow INFO:
	 [Node] Cached "7157e2e3c16071cf6fe61ec267976d25" - collecting precomputed outputs
200619-21:59:37,354 nipype.workflow INFO:
	 [Node] "7157e2e3c16071cf6fe61ec267976d25" found cached.
200619-21:59:37,376 nipype.workflow INFO:
	 [Node] Setting-up "985381117875801ce9efa3dc8ec12bff" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-Level1Design/985381117875801ce9efa3dc8ec12bff".
200619-21:59:37,385 nipype.workflow INFO:
	 [Node] Cached "985381117875801ce9efa3dc8ec12bff" - collecting precomputed outputs
200619-21:59:37,387 nipype.workflow INFO:
	 [Node] "985381117875801ce9efa3dc8ec12bff" found cached.
200619-21:59:37,394 nipy

200619-21:59:37,700 nipype.workflow INFO:
	 [Node] Setting-up "d3833dd0618225b55b25b6d5007fdbdf" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FEATModel/d3833dd0618225b55b25b6d5007fdbdf".
200619-21:59:37,703 nipype.workflow INFO:
	 [Node] Cached "d3833dd0618225b55b25b6d5007fdbdf" - collecting precomputed outputs
200619-21:59:37,704 nipype.workflow INFO:
	 [Node] "d3833dd0618225b55b25b6d5007fdbdf" found cached.
200619-21:59:37,710 nipype.workflow INFO:
	 [Node] Setting-up "5428928e55f3e871d517bbfc887f348f" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-maths-ApplyMask/5428928e55f3e871d517bbfc887f348f".
200619-21:59:37,712 nipype.workflow INFO:
	 [Node] Cached "5428928e55f3e871d517bbfc887f348f" - collecting precomputed outputs
200619-21:59:37,713 nipype.workflow INFO:
	 [Node] "5428928e55f3e871d517bbfc887f348f" found cached.
200619-21:59:37,722 nipype.w

200619-21:59:38,49 nipype.workflow INFO:
	 [Node] Setting-up "79f8872a6575c0e1f3dec027bd4d7cd8" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FILMGLS/79f8872a6575c0e1f3dec027bd4d7cd8".
200619-21:59:38,55 nipype.workflow INFO:
	 [Node] Cached "79f8872a6575c0e1f3dec027bd4d7cd8" - collecting precomputed outputs
200619-21:59:38,56 nipype.workflow INFO:
	 [Node] "79f8872a6575c0e1f3dec027bd4d7cd8" found cached.
200619-21:59:38,92 nipype.workflow INFO:
	 [Node] Setting-up "9e3041b89e343e8f980d525cc110ed08" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-utils-ExtractROI/9e3041b89e343e8f980d525cc110ed08".
200619-21:59:38,94 nipype.workflow INFO:
	 [Node] Cached "9e3041b89e343e8f980d525cc110ed08" - collecting precomputed outputs
200619-21:59:38,95 nipype.workflow INFO:
	 [Node] "9e3041b89e343e8f980d525cc110ed08" found cached.
200619-21:59:38,119 nipype.workflow

200619-21:59:38,454 nipype.workflow INFO:
	 [Node] Setting-up "c31ca59488fbfed58bc99128eb7b7c89" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-Level1Design/c31ca59488fbfed58bc99128eb7b7c89".
200619-21:59:38,463 nipype.workflow INFO:
	 [Node] Cached "c31ca59488fbfed58bc99128eb7b7c89" - collecting precomputed outputs
200619-21:59:38,463 nipype.workflow INFO:
	 [Node] "c31ca59488fbfed58bc99128eb7b7c89" found cached.
200619-21:59:38,470 nipype.workflow INFO:
	 [Node] Setting-up "2911e494a65111b287f48de1e09656ce" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FEATModel/2911e494a65111b287f48de1e09656ce".
200619-21:59:38,472 nipype.workflow INFO:
	 [Node] Cached "2911e494a65111b287f48de1e09656ce" - collecting precomputed outputs
200619-21:59:38,473 nipype.workflow INFO:
	 [Node] "2911e494a65111b287f48de1e09656ce" found cached.
200619-21:59:38,479 nipyp

200619-21:59:38,777 nipype.workflow INFO:
	 [Node] Setting-up "5f1656a0ab75ed21d739f5d102f965ad" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-maths-ApplyMask/5f1656a0ab75ed21d739f5d102f965ad".
200619-21:59:38,778 nipype.workflow INFO:
	 [Node] Cached "5f1656a0ab75ed21d739f5d102f965ad" - collecting precomputed outputs
200619-21:59:38,779 nipype.workflow INFO:
	 [Node] "5f1656a0ab75ed21d739f5d102f965ad" found cached.
200619-21:59:38,787 nipype.workflow INFO:
	 [Node] Setting-up "053600af74fe2d59409bf24119d0796c" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FILMGLS/053600af74fe2d59409bf24119d0796c".
200619-21:59:38,793 nipype.workflow INFO:
	 [Node] Cached "053600af74fe2d59409bf24119d0796c" - collecting precomputed outputs
200619-21:59:38,793 nipype.workflow INFO:
	 [Node] "053600af74fe2d59409bf24119d0796c" found cached.
200619-21:59:38,834 nipype.wor

...

200619-21:59:48,654 nipype.workflow INFO:
	 [Node] Setting-up "5515e247285a21c362313cd48571c4b1" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-Level1Design/5515e247285a21c362313cd48571c4b1".
200619-21:59:48,663 nipype.workflow INFO:
	 [Node] Cached "5515e247285a21c362313cd48571c4b1" - collecting precomputed outputs
200619-21:59:48,664 nipype.workflow INFO:
	 [Node] "5515e247285a21c362313cd48571c4b1" found cached.
200619-21:59:48,672 nipype.workflow INFO:
	 [Node] Setting-up "9d33860610d8eb5816af3e0256b725ac" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FEATModel/9d33860610d8eb5816af3e0256b725ac".
200619-21:59:48,676 nipype.workflow INFO:
	 [Node] Cached "9d33860610d8eb5816af3e0256b725ac" - collecting precomputed outputs
200619-21:59:48,677 nipype.workflow INFO:
	 [Node] "9d33860610d8eb5816af3e0256b725ac" found cached.
200619-21:59:48,683 nipyp

200619-21:59:49,70 nipype.workflow INFO:
	 [Node] Setting-up "138c87a1b72645f1af9cf44e1ed6cc1a" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-maths-ApplyMask/138c87a1b72645f1af9cf44e1ed6cc1a".
200619-21:59:49,73 nipype.workflow INFO:
	 [Node] Cached "138c87a1b72645f1af9cf44e1ed6cc1a" - collecting precomputed outputs
200619-21:59:49,74 nipype.workflow INFO:
	 [Node] "138c87a1b72645f1af9cf44e1ed6cc1a" found cached.
200619-21:59:49,88 nipype.workflow INFO:
	 [Node] Setting-up "d50db17c2f1e4e57d9a0f7d57783af0a" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FILMGLS/d50db17c2f1e4e57d9a0f7d57783af0a".
200619-21:59:49,98 nipype.workflow INFO:
	 [Node] Cached "d50db17c2f1e4e57d9a0f7d57783af0a" - collecting precomputed outputs
200619-21:59:49,99 nipype.workflow INFO:
	 [Node] "d50db17c2f1e4e57d9a0f7d57783af0a" found cached.
200619-21:59:49,155 nipype.workflow 

200619-21:59:49,479 nipype.workflow INFO:
	 [Node] Setting-up "6c3b3d2f4d2c7eb6f054778ff7006526" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-utils-ExtractROI/6c3b3d2f4d2c7eb6f054778ff7006526".
200619-21:59:49,481 nipype.workflow INFO:
	 [Node] Cached "6c3b3d2f4d2c7eb6f054778ff7006526" - collecting precomputed outputs
200619-21:59:49,482 nipype.workflow INFO:
	 [Node] "6c3b3d2f4d2c7eb6f054778ff7006526" found cached.
200619-21:59:49,504 nipype.workflow INFO:
	 [Node] Setting-up "7512bb253b9518994fb8350fe48f1d9d" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-Level1Design/7512bb253b9518994fb8350fe48f1d9d".
200619-21:59:49,513 nipype.workflow INFO:
	 [Node] Cached "7512bb253b9518994fb8350fe48f1d9d" - collecting precomputed outputs
200619-21:59:49,514 nipype.workflow INFO:
	 [Node] "7512bb253b9518994fb8350fe48f1d9d" found cached.
200619-21:59:49,521 nipy

200619-21:59:49,892 nipype.workflow INFO:
	 [Node] Setting-up "9786e6ec25c1a36292471cfb938ef805" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FEATModel/9786e6ec25c1a36292471cfb938ef805".
200619-21:59:49,895 nipype.workflow INFO:
	 [Node] Cached "9786e6ec25c1a36292471cfb938ef805" - collecting precomputed outputs
200619-21:59:49,896 nipype.workflow INFO:
	 [Node] "9786e6ec25c1a36292471cfb938ef805" found cached.
200619-21:59:49,902 nipype.workflow INFO:
	 [Node] Setting-up "cff02a379c808de0173bf53eee5e9e72" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-maths-ApplyMask/cff02a379c808de0173bf53eee5e9e72".
200619-21:59:49,904 nipype.workflow INFO:
	 [Node] Cached "cff02a379c808de0173bf53eee5e9e72" - collecting precomputed outputs
200619-21:59:49,905 nipype.workflow INFO:
	 [Node] "cff02a379c808de0173bf53eee5e9e72" found cached.
200619-21:59:49,914 nipype.w

200619-21:59:50,291 nipype.workflow INFO:
	 [Node] Setting-up "7464b53074a2e3fe452d8255e01af03f" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FILMGLS/7464b53074a2e3fe452d8255e01af03f".
200619-21:59:50,298 nipype.workflow INFO:
	 [Node] Cached "7464b53074a2e3fe452d8255e01af03f" - collecting precomputed outputs
200619-21:59:50,299 nipype.workflow INFO:
	 [Node] "7464b53074a2e3fe452d8255e01af03f" found cached.


In [12]:
for icond in range(len(task_cond)):
    for isub, iimg in zmaps.items(): 
        smooth_zmap = nilearn.image.smooth_img(iimg[icond], 6)
        if fig_plv == 1: threshold = None
        else: _, threshold = map_threshold(smooth_zmap, alpha = fig_plv, height_control = fig_cor)
    
        if fig_in_nb == 'no': 
            nilearn.plotting.plot_glass_brain(
                smooth_zmap, display_mode = 'ortho', colorbar = True, threshold = threshold, plot_abs = False, 
                title = '%s_%s_%s_%s_%s'%(task_name, task_cond[icond], fig_cor, str(fig_plv), isub), 
                output_file = os.path.join(figure_dir, '%s_%s_%s_%s_%s.png'%(task_name, task_cond[icond], fig_cor, str(fig_plv), isub)))
        elif fig_in_nb == 'yes': 
            nilearn.plotting.plot_glass_brain(
                smooth_zmap, display_mode = 'ortho', colorbar = True, threshold = threshold, plot_abs = False, 
                title = '%s_%s_%s_%s_%s'%(task_name, task_cond[icond], fig_cor, str(fig_plv), isub), 
                axes = [1, 1, 0.8, 0.8])